## Import and set constants

In [32]:
import glob
from dataclasses import dataclass
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from scipy.interpolate import InterpolatedUnivariateSpline

import math

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

INPUT_PATH = "sdc2023"

WGS84_SEMI_MAJOR_AXIS = 6378137.0
WGS84_SEMI_MINOR_AXIS = 6356752.314245
WGS84_SQUARED_FIRST_ECCENTRICITY  = 6.69437999013e-3
WGS84_SQUARED_SECOND_ECCENTRICITY = 6.73949674226e-3

HAVERSINE_RADIUS = 6_371_000

## Load Device

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
device

device(type='cuda')

## Class to manage converting between BLH and GNSS
Needed since ground truth is in BLH (latitude, longitude, height) and GNSS is in ECEF (x, y z with COM as origin)

In [34]:
@dataclass
class ECEF:
    x: np.ndarray
    y: np.ndarray
    z: np.ndarray

    def to_numpy(self):
        return np.stack([self.x, self.y, self.z], axis=0)

    @staticmethod
    def from_numpy(pos: np.ndarray):
        x, y, z = [np.squeeze(w) for w in np.split(pos, 3, axis=-1)]
        return ECEF(x=x, y=y, z=z)

@dataclass
class BLH:
    lat: np.ndarray
    lng: np.ndarray
    hgt: np.ndarray = 0
        

def ECEF_to_BLH(ecef: ECEF) -> BLH:
    """
    Convert Earth-Centered, Earth-Fixed (ECEF) coordinates to geodetic coordinates (latitude, longitude, height).

    Args:
        ecef (ECEF): ECEF coordinates (x, y, z).

    Returns:
        BLH: Geodetic coordinates (latitude, longitude, height).
    """
    a = WGS84_SEMI_MAJOR_AXIS
    b = WGS84_SEMI_MINOR_AXIS
    e2  = WGS84_SQUARED_FIRST_ECCENTRICITY
    e2_ = WGS84_SQUARED_SECOND_ECCENTRICITY
    x = ecef.x
    y = ecef.y
    z = ecef.z
    r = np.sqrt(x**2 + y**2)
    t = np.arctan2(z * (a/b), r)
    B = np.arctan2(z + (e2_*b)*np.sin(t)**3, r - (e2*a)*np.cos(t)**3)
    L = np.arctan2(y, x)
    n = a / np.sqrt(1 - e2*np.sin(B)**2)
    H = (r / np.cos(B)) - n
    # convert to degrees
    B = np.rad2deg(B)
    L = np.rad2deg(L)
    return BLH(lat=B, lng=L, hgt=H)

def haversine_distance(blh_1: BLH, blh_2: BLH) -> np.ndarray:
    """
    Calculate the haversine distance between two sets of points on the Earth's surface.

    Args:
        blh_1 (BLH): Geodetic coordinates of the first point.
        blh_2 (BLH): Geodetic coordinates of the second point.

    Returns:
        np.ndarray: Haversine distance between the two sets of points.
    """
    lat_1 = np.deg2rad(blh_1.lat)
    lng_1 = np.deg2rad(blh_1.lng)
    lat_2 = np.deg2rad(blh_2.lat)
    lng_2 = np.deg2rad(blh_2.lng)
    
    dlat = lat_2 - lat_1
    dlng = lng_2 - lng_1
    a = np.sin(dlat/2)**2 + np.cos(lat_1) * np.cos(lat_2) * np.sin(dlng/2)**2
    dist = 2 * HAVERSINE_RADIUS * np.arcsin(np.sqrt(a))
    return dist

def pandas_haversine_distance(df1: pd.DataFrame, df2: pd.DataFrame) -> np.ndarray:
    """
    Calculate the haversine distance between two sets of geodetic coordinates.

    Args:
        df1 (pd.DataFrame): First set of geodetic coordinates.
        df2 (pd.DataFrame): Second set of geodetic coordinates.
    
    Returns:
        np.ndarray: Haversine distance between the two sets of coordinates.
    """
    blh1 = BLH(
        lat=df1['LatitudeDegrees'].to_numpy(),
        lng=df1['LongitudeDegrees'].to_numpy(),
        hgt=0,
    )
    blh2 = BLH(
        lat=df2['LatitudeDegrees'].to_numpy(),
        lng=df2['LongitudeDegrees'].to_numpy(),
        hgt=0,
    )
    return haversine_distance(blh1, blh2)


In [5]:
def ecef_to_lat_lng(tripID: str, gnss_df: pd.DataFrame, UnixTimeMillis: pd.Series | np.ndarray) -> pd.DataFrame:
    """
    Convert ECEF coordinates to geodetic coordinates (latitude, longitude).

    Args:
        tripID (str): Trip ID.
        gnss_df (pd.DataFrame): GNSS data.
        UnixTimeMillis (pd.Series | np.ndarray): Unix time in milliseconds.
    
    Returns:
        pd.DataFrame: Geodetic coordinates (latitude, longitude).
    """
    ecef_columns = ['WlsPositionXEcefMeters', 'WlsPositionYEcefMeters', 'WlsPositionZEcefMeters']
    columns = ['utcTimeMillis'] + ecef_columns
    ecef_df = (gnss_df.drop_duplicates(subset='utcTimeMillis')[columns]
               .dropna().reset_index(drop=True))
    ecef = ECEF.from_numpy(ecef_df[ecef_columns].to_numpy())
    blh  = ECEF_to_BLH(ecef)

    TIME = ecef_df['utcTimeMillis'].to_numpy()
    lat = InterpolatedUnivariateSpline(TIME, blh.lat, ext=3)(UnixTimeMillis)
    lng = InterpolatedUnivariateSpline(TIME, blh.lng, ext=3)(UnixTimeMillis)
    return pd.DataFrame({
#         'tripId' : tripID,
        'utcTimeMillis'   : UnixTimeMillis,
        'LatitudeDegrees'  : np.degrees(lat),
        'LongitudeDegrees' : np.degrees(lng),
    })

def calc_score(pred_df: pd.DataFrame, gt_df: pd.DataFrame) -> float:
    """
    Calculate the score of the predicted trajectory.

    Args:
        pred_df (pd.DataFrame): Predicted trajectory.
        gt_df (pd.DataFrame): Ground truth trajectory.

    Returns:
        float: Score of the predicted trajectory.
    """
    d = pandas_haversine_distance(pred_df, gt_df)
    score = np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])    
    return score


In [6]:
def print_comparison(lat, lng, gt_lat, gt_lng):
    for lat_val, lng_val, gt_lat_val, gt_lng_val in zip(lat, lng, gt_lat, gt_lng):
        print(f'Pred: ({lat_val:<12.7f}, {lng_val:<12.7f}) Ground Truth: ({gt_lat_val:<12.7f}, {gt_lng_val:<12.7f})')
        
def print_batch(amnt: int, lat_arr: np.ndarray, lng_arr: np.ndarray, gt_lat_arr: np.ndarray, gt_lng_arr: np.ndarray):
    for batch in range(amnt):
        print(f'Val data {batch}')
        print_comparison(lat_arr[batch], lng_arr[batch], gt_lat_arr[batch], gt_lng_arr[batch])

## Loading Data

In [7]:
%%capture --no-stdout

pred_dfs  = []
gt_dfs = []



for dirname in sorted(glob.glob(f'{INPUT_PATH}/train/*/*')):
    drive, phone = dirname.split('/')[-2:]
    tripID  = f'{drive}/{phone}'
    gnss_df = pd.read_csv(f'{dirname}/device_gnss.csv')
    gt_df   = pd.read_csv(f'{dirname}/ground_truth.csv')
    
    info_cols = ['IonosphericDelayMeters', 'TroposphericDelayMeters']
    columns = ['utcTimeMillis'] + info_cols
    info_df = (gnss_df.drop_duplicates(subset='utcTimeMillis')[columns].fillna(0).reset_index(drop=True))
    
    for col in info_cols:
        info_df[col] = info_df[col].fillna((info_df[col].bfill() + info_df[col].ffill()) / 2)
        
    pred_df = ecef_to_lat_lng(tripID, gnss_df, gt_df['UnixTimeMillis'])
    pred_df = pd.merge(pred_df, info_df, on='utcTimeMillis', how='left')
    gt_df   = gt_df[['LatitudeDegrees', 'LongitudeDegrees']]
    print(tripID)
#     print(pred_df.shape)
#     print(gt_df.shape)
    pred_dfs.append(pred_df)
    gt_dfs.append(gt_df)

2020-06-25-00-34-us-ca-mtv-sb-101/pixel4
2020-06-25-00-34-us-ca-mtv-sb-101/pixel4xl
2020-07-08-22-28-us-ca/pixel4
2020-07-08-22-28-us-ca/pixel4xl
2020-07-17-22-27-us-ca-mtv-sf-280/pixel4
2020-07-17-23-13-us-ca-sf-mtv-280/pixel4
2020-07-17-23-13-us-ca-sf-mtv-280/pixel4xl
2020-08-04-00-19-us-ca-sb-mtv-101/pixel4
2020-08-04-00-20-us-ca-sb-mtv-101/pixel4xl
2020-08-04-00-20-us-ca-sb-mtv-101/pixel5
2020-08-13-21-41-us-ca-mtv-sf-280/pixel4
2020-08-13-21-41-us-ca-mtv-sf-280/pixel4xl
2020-08-13-21-42-us-ca-mtv-sf-280/pixel5
2020-12-10-22-17-us-ca-sjc-c/mi8
2020-12-10-22-52-us-ca-sjc-c/mi8
2020-12-10-22-52-us-ca-sjc-c/pixel4
2020-12-10-22-52-us-ca-sjc-c/pixel4xl
2020-12-10-22-52-us-ca-sjc-c/pixel5
2021-01-04-21-50-us-ca-e1highway280driveroutea/mi8
2021-01-04-21-50-us-ca-e1highway280driveroutea/pixel4
2021-01-04-21-50-us-ca-e1highway280driveroutea/pixel5
2021-01-04-22-40-us-ca-mtv-a/mi8
2021-01-04-22-40-us-ca-mtv-a/pixel4
2021-01-04-22-40-us-ca-mtv-a/pixel5
2021-01-05-21-12-us-ca-mtv-d/mi8
2021-0

## Define Baseline Transformer

In [8]:
class PositionalEncoding(torch.nn.Module):
    """
    This module injects some information about the relative or absolute position of the tokens in the sequence.
    The positional encodings have the same dimension as the embeddings, so that the two can be summed. Here, we use
    sine and cosine functions of different frequencies.

    Args:
        d_model (int): The dimension of the model (i.e., the size of the input embeddings).
        dropout (float): Dropout rate to apply after adding the positional encodings.
        max_len (int): The maximum length of the input sequences for which to precompute positional encodings.

    Attributes:
        dropout (torch.nn.Dropout): Dropout layer to apply after adding positional encodings.
        pe (torch.Tensor): Precomputed positional encodings.
    """
    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        """
        Initializes the PositionalEncoding module.

        Args:
            d_model (int): The dimension of the model (i.e., the size of the input embeddings).
            dropout (float): Dropout rate to apply after adding the positional encodings.
            max_len (int): The maximum length of the input sequences for which to precompute positional encodings.
        """
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Adds positional encodings to the input tensor and applies dropout.

        Args:
            x (torch.Tensor): Input tensor of shape (seq_len, batch_size, embedding_dim).

        Returns:
            torch.Tensor: Tensor with positional encodings added, followed by dropout.
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)
    
# time sequence encoder
class TransformerEncoder(torch.nn.Module):
    """
    Transformer Encoder, which consists of an input linear layer to upscale the input 
    dimension to the model dimension, a positional encoding layer, a stack of Transformer encoder layers, and 
    a final fully connected (fc) layer for output transformation.

    Args:
        config (object): A configuration object containing the hyperparameters.
    
    Attributes:
        config (object): The configuration object.
        upscale (torch.nn.Linear): Linear layer to upscale input dimension to model dimension.
        pos_encoder (PositionalEncoding): Positional encoding layer.
        transformer (torch.nn.TransformerEncoder): Stack of Transformer encoder layers.
        fc (torch.nn.Sequential): Fully connected layers for output transformation.
    """
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.upscale = torch.nn.Linear(config.input_dim, config.d_model) 
        self.pos_encoder = PositionalEncoding(config.d_model, config.dropout, config.max_seq_len)
        self.transformer = torch.nn.TransformerEncoder(
            torch.nn.TransformerEncoderLayer(
                d_model=config.d_model,
                nhead=config.nhead,
                dim_feedforward=config.dim_feedforward,
                dropout=config.dropout,
                activation=config.activation,
                batch_first=True
            ),
            num_layers=config.num_layers
        )
        self.fc = torch.nn.Sequential()
        for i, num_neurons in enumerate(config.fc_layers[:-1]):
            self.fc.add_module(
                f'fc_{i}',
                torch.nn.Linear(num_neurons, config.fc_layers[i+1])
            )
            if i < len(config.fc_layers) - 1:
                self.fc.add_module(
                    f'relu_{i}',
                    torch.nn.ReLU()
                )
        
        
    def forward(self, x):
        x = self.upscale(x)
        x = self.pos_encoder(x)
        x = self.transformer(x)
        x = self.fc(x)
        return x

## Define baseline LSTM

In [9]:
class LSTMEncoder(torch.nn.Module):
    """
    LSTM Encoder, which consists of an input linear layer to upscale the input 
    dimension to the model dimension, a stack of LSTM layers, and a final fully connected (fc) layer for output transformation.

    Args:
        config (object): A configuration object containing the hyperparameters.
    
    Attributes:
        config (object): The configuration object.
        upscale (torch.nn.Linear): Linear layer to upscale input dimension to model dimension.
        lstm (torch.nn.LSTM): Stack of LSTM layers.
        fc (torch.nn.Sequential): Fully connected layers for output transformation.
    """
    
    def __init__(self, config):
        """
        Initializes the LSTMEncoder module.

        Args:
            config (object): A configuration object containing the hyperparameters for the LSTM Encoder.
        """
        super().__init__()
        self.config = config
        
        # Linear layer to upscale the input dimension to the model dimension
        self.upscale = torch.nn.Linear(config.input_dim, config.d_model)
        
        # Stack of LSTM layers
        self.lstm = torch.nn.LSTM(
            input_size=config.d_model,
            hidden_size=config.d_model,
            num_layers=config.num_layers,
            dropout=config.dropout,
            batch_first=True
        )
        
        # Fully connected layers for output transformation
        self.fc = torch.nn.Sequential()
        for i, num_neurons in enumerate(config.fc_layers[:-1]):
            self.fc.add_module(
                f'fc_{i}',
                torch.nn.Linear(num_neurons, config.fc_layers[i+1])
            )
            if i < len(config.fc_layers) - 1:
                self.fc.add_module(
                    f'relu_{i}',
                    torch.nn.ReLU()
                )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Defines the forward pass of the LSTMEncoder.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, seq_len, input_dim).

        Returns:
            torch.Tensor: Output tensor after passing through the LSTM Encoder.
        """
        # Upscale the input dimension to the model dimension
        x = self.upscale(x)
        
        # Pass through the stack of LSTM layers
        x, _ = self.lstm(x)
        
        # Transform the output through fully connected layers
        x = self.fc(x)
        
        return x

## Declare Model

In [10]:
class Config:
    """
    Configuration class to hold the hyperparameters and other settings.

    Args:
        config_dict (dict): A dictionary containing the hyperparameters and other settings.
    
    Attributes:
        input_dim (int): The dimension of the input features.
        d_model (int): The dimension of the model (i.e., the size of the input embeddings).
        nhead (int): The number of heads in the multiheadattention models.
        dim_feedforward (int): The dimension of the feedforward network model.
        dropout (float): The dropout value.
        activation (str): The activation function of intermediate layer, relu or gelu.
        num_layers (int): The number of sub-encoder-layers in the encoder.
        fc_layers (list[int]): The number of neurons in the fully connected layers.
        output_dim (int): The dimension of the output features.
        max_seq_len (int): The maximum sequence length.
        val_split (float): The validation split ratio.
    """
    def __init__(self, config_dict):
        self.input_dim = config_dict.get('input_dim')
        self.d_model = config_dict.get('d_model')
        self.nhead = config_dict.get('nhead')
        self.dim_feedforward = config_dict.get('dim_feedforward')
        self.dropout = config_dict.get('dropout')
        self.activation = config_dict.get('activation')
        self.num_layers = config_dict.get('num_layers')
        self.fc_layers = config_dict.get('fc_layers')
        self.output_dim = config_dict.get('output_dim')
        self.max_seq_len = config_dict.get('max_seq_len')
        self.val_split = config_dict.get('val_split')

config_dict = {
    "input_dim": 4, 
    "d_model": 8,
    "nhead": 4,
    "dim_feedforward": 128,
    "dropout": 0.1,
    "activation": "relu",
    "num_layers": 6,
    "output_dim": 2, 
    "max_seq_len": 3500,
    "val_split": 0.05,
}
# For fc layers divide the d_model by 2 until it reaches the output_dim
fc_layers = [config_dict["d_model"]]
while fc_layers[-1] > config_dict["output_dim"]:
    fc_layers.append(fc_layers[-1] // 2)

fc_layers[-1] = config_dict["output_dim"]
config_dict["fc_layers"] = fc_layers

config = Config(config_dict)

In [11]:
model = TransformerEncoder(config)
model.to(device)

TransformerEncoder(
  (upscale): Linear(in_features=4, out_features=8, bias=True)
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer): TransformerEncoder(
    (layers): ModuleList(
      (0-5): 6 x TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=8, out_features=8, bias=True)
        )
        (linear1): Linear(in_features=8, out_features=128, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=128, out_features=8, bias=True)
        (norm1): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((8,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): Dropout(p=0.1, inplace=False)
      )
    )
  )
  (fc): Sequential(
    (fc_0): Linear(in_features=8, out_features=4, bias=True)
    (relu_0): ReLU()
    (fc_1): Linear(in_features=4,

## Define Dataset Class

In [12]:

class GNSSDataset(torch.utils.data.Dataset):
    """
    This class represents a custom dataset for Global Navigation Satellite System (GNSS) data.
    It processes prediction and ground truth dataframes, pads sequences to a maximum length,
    and computes the residuals between predictions and ground truth positions.
    Args:
        pred_dfs (list of pandas.DataFrame): List of dataframes containing prediction data.
        gt_dfs (list of pandas.DataFrame): List of dataframes containing ground truth data.
    
    Attributes:
        pred_dfs (list of pandas.DataFrame): List of dataframes containing prediction data.
        gt_dfs (list of pandas.DataFrame): List of dataframes containing ground truth data.
        sequences (numpy.ndarray): Numpy array of padded prediction sequences.
        labels (numpy.ndarray): Numpy array of residuals (ground truth - prediction).
    """
    def __init__(self, pred_dfs, gt_dfs):
        """
        Initializes the GNSSDataset.
        Args:
            pred_dfs (list of pandas.DataFrame): List of dataframes containing prediction data.
            gt_dfs (list of pandas.DataFrame): List of dataframes containing ground truth data.
        """
        self.pred_dfs = pred_dfs
        self.gt_dfs = gt_dfs
        self.sequences = []
        self.labels = []
        
        for pred_df in self.pred_dfs:
            x_np = pred_df[['LatitudeDegrees', 'LongitudeDegrees', 'IonosphericDelayMeters', 'TroposphericDelayMeters']].to_numpy()
            ## pad to max sequence length
            pad = np.zeros((config.max_seq_len - x_np.shape[0], x_np.shape[1]))
            x_np = np.vstack([x_np, pad])
            #x_np = x_np/180
            self.sequences.append(x_np)
            
        for gt_df in self.gt_dfs:
            y_np = gt_df[['LatitudeDegrees', 'LongitudeDegrees']].to_numpy()
            ## pad to max sequence length
            pad = np.zeros((config.max_seq_len - y_np.shape[0], y_np.shape[1]))
            y_np = np.vstack([y_np, pad])
            #y_np = y_np/180
            self.labels.append(y_np)

        self.sequences = np.array(self.sequences, dtype=np.float32)
        self.labels = np.array(self.labels,  dtype=np.float32)
        
        self.labels = self.labels - self.sequences[:, :, :2] # just the residuals
        
        print("seq and label shapes")
        print(self.sequences.shape)
        print(self.labels.shape)

    def __getitem__(self, i):
        """
        Retrieves the sequence and label at index i.
        Args:
            i (int): Index of the data to retrieve.
        Returns:
            tuple: (sequence, label) at index i.
        """
        return self.sequences[i], self.labels[i]
    def __len__(self):
        """
        Returns the number of sequences in the dataset.
        Returns:
            int: Number of sequences in the dataset.
        """
        return self.sequences.shape[0]

## Initialize Dataset

In [13]:
import sklearn
from sklearn.model_selection import train_test_split

In [14]:
print(len(pred_dfs)*config.val_split)

7.800000000000001


In [15]:
# train test split into training and validation
X_train, X_val, y_train, y_val = train_test_split(pred_dfs, gt_dfs, 
                            test_size=config.val_split, random_state=2)


train_dataset = GNSSDataset(X_train, y_train)
val_dataset = GNSSDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True)

seq and label shapes
(148, 3500, 4)
(148, 3500, 2)
seq and label shapes
(8, 3500, 4)
(8, 3500, 2)


## Train the Model

In [16]:
epochs = 100 # number of epochs
n_eval = 10 # evaluate every n_eval epochs
lr = 0.0001 # learning rate

In [35]:
# Initalize optimizer (for gradient descent) and loss function
optimizer = torch.optim.Adam(params = model.parameters(), lr = lr)
loss_fn = torch.nn.MSELoss()

step = 0

PATH = "model.pt"
best_loss = 99999999 #high number

# Regularization strength
lambda_l1 = 0.1
lambda_l2 = 0.25

for epoch in range(epochs):
    print(f"Epoch {epoch + 1} of {epochs}")

    # Loop over each batch in the dataset
    for batch in tqdm(train_loader):
        
        optimizer.zero_grad() # If not, the gradients would sum up over each iteration

        # Unpack the data and labels
        features, labels = batch
        
        features = features.to(device)
        labels = labels.to(device)

        # Forward propagate

        outputs = model(features)

        # Backpropagation and gradient descent            

        loss = loss_fn(outputs, labels)
        # L1 regularization
        l1_reg = sum(param.abs().sum() for param in model.parameters())
        # L2 regularization
        l2_reg = sum(param.pow(2).sum() for param in model.parameters())

        # Add the regularization terms to the loss
        loss += lambda_l1 * l1_reg + lambda_l2 * l2_reg

        loss.backward()
        optimizer.step()
        
        print(f'Loss/train {loss}')

        # Periodically evaluate our model + log to Tensorboard
        if step % n_eval == 0:
            # Compute training loss and metrics
            
            model.eval()
            mean_dist = 0
            mean_score = 0
            count = 0
            val_loss = 0
            for features, labels in val_loader:
#                 print(features.shape)
#                 print(features[0, :20])
                features = features.to(device)
                labels = labels.to(device)
                
                val_pred = model(features)
                
                val_loss = loss_fn(val_pred, labels)
                
                features = features.detach().cpu()# * 180
                val_pred = val_pred.detach().cpu()# * 180
                labels = labels.detach().cpu()# * 180
                print(val_pred.shape)
                pred_lats = val_pred[:, :, 0] + features[:, :, 0]
                pred_lngs = val_pred[:, :, 1] + features[:, :, 1]
                gt_lats = labels[:, :, 0] + features[:, :, 0]
                gt_lns = labels[:, :, 1] + features[:, :, 1]
                
                start = 60
                cutoff = 65
                # print some samples
                print_batch(1, pred_lats[:, start:cutoff], pred_lngs[:, start:cutoff], gt_lats[:, start:cutoff], gt_lns[:, start:cutoff]) 
                
                # Calculate score according to kaggle, height not necessary for distance
                blh1 = BLH(pred_lats, pred_lngs, hgt = 0)
                blh2 = BLH(gt_lats, gt_lns, hgt = 0)
                d = haversine_distance(blh1, blh2)
                mean_score += np.mean([np.quantile(d, 0.50), np.quantile(d, 0.95)])     
                mean_dist += d.mean()  
                
                count += 1
                
                features = features.cpu()
                labels = labels.cpu()
                
            if(val_loss < best_loss):
                best_loss = val_loss
                
                torch.save({
                            'epoch': epoch,
                            'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'loss': val_loss,
                            }, PATH)

                
            print(f'Val mean dist {mean_dist}')
            print(f'Val mean score {mean_score}')
            print(f'Loss/val {val_loss}')

            #turn on training, evaluate turns off training
            model.train()

        step += 1


Epoch 1 of 100

  0%|          | 0/5 [00:00<?, ?it/s]

Loss/train 309.1878356933594
torch.Size([8, 3500, 2])
Val data 0
Pred: (37.3089371  , -121.9904861) Ground Truth: (37.2889252  , -121.9904709)
Pred: (37.3089333  , -121.9905472) Ground Truth: (37.2889252  , -121.9905243)
Pred: (37.3089294  , -121.9906845) Ground Truth: (37.2889290  , -121.9905777)
Pred: (37.3089027  , -121.9906921) Ground Truth: (37.2889328  , -121.9906387)
Pred: (37.3089371  , -121.9907074) Ground Truth: (37.2889404  , -121.9906998)


/tmp/ipykernel_349/2214136882.py:61: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  lat_1 = np.deg2rad(blh_1.lat)
/tmp/ipykernel_349/2214136882.py:62: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  lng_1 = np.deg2rad(blh_1.lng)
/tmp/ipykernel_349/2214136882.py:63: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  lat_2 = np.deg2rad(blh_2.lat)
/tmp/ipykernel_349/2214136882.py:64: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  lng_2 = np.deg2rad(blh_2.lng)
/tmp/ipykernel_349/2214136882.py:68: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  a = n

Val mean dist 13901.3984375
Val mean score 11018.6513671875
Loss/val 0.18737797439098358
